In [1]:
pip install faiss-cpu sentence-transformers transformers newspaper3k flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 73.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
 

In [7]:
from newspaper import Article
import json
import faiss
from sentence_transformers import SentenceTransformer
import json

In [4]:
urls = [
    'https://edition.cnn.com/2024/06/20/business/ai-jobs-workers-replacing/index.html',
    'https://www.bbc.com/news/articles/cd114dpnqxwo',
    'https://www.bbc.com/news/articles/c8003dd8jzeo',
    'https://www.bbc.com/news/articles/c2jj5mee1pmo'
]

articles = []
for url in urls:
    try:
        article = Article(url)
        article.download()
        article.parse()
        articles.append({'title': article.title, 'text': article.text})
    except Exception as e:
        print(f"Exception: {url}\n{e}")

with open('articles.json', 'w') as f:
    json.dump(articles, f)

In [8]:
with open('articles.json', 'r') as f:
    articles = json.load(f)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

texts = [article['text'] for article in articles]
embeddings = model.encode(texts)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
faiss.write_index(index, 'faiss_index')

with open('articles.json', 'w') as f:
    json.dump(articles, f)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
def search_documents(query, top_k=2):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = [articles[idx] for idx in indices[0]]
    return results

In [10]:
from transformers import pipeline
qa_pipeline = pipeline("question-answering")

def generate_answer(question, context):
    response = qa_pipeline(question=question, context=context)
    return response['answer']

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [14]:
pip install telebot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.9/244.9 kB 5.0 MB/s eta 0:00:00


In [18]:
import telebot

TELEGRAM_TOKEN = "AAGQIchov-IL-tc99ZplKWMHFmk-H9cr-34"

bot = telebot.TeleBot(TELEGRAM_TOKEN)

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Привет! Задай мне вопрос, и я постараюсь найти ответ.")

@bot.message_handler(func=lambda message: True)
def handle_message(message):
    question = message.text
    relevant_docs = search_documents(question)
    combined_context = " ".join([doc['text'] for doc in relevant_docs])
    answer = generate_answer(question, combined_context)
    bot.reply_to(message, answer)

bot.polling()